In [1]:
import urllib
import requests
from pandas.io.json import json_normalize
import json
import pandas as pd
pd.options.mode.chained_assignment = None
import datetime
import time
import math
import numpy as np

from neural_network import NeuralNetwork

"""
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Activation, Dense
from keras.models import Sequential
"""


def generate_item_records_from_summary():
    """ Generate item records using the summary.json link from the OSBuddy API """

    df = pd.read_json(path_or_buf='https://rsbuddy.com/exchange/summary.json',orient='index', convert_axes=True)
    df = df[['id','name','buy_average','buy_quantity','sell_average','sell_quantity','overall_average','overall_quantity']]
    data = df.sort_values(by=['id']).reset_index()
    data = data.drop(labels='index',axis=1)

    #Output item id/name pairs to a csv file
    item_key = data[['id', 'name']]
    file_name = './item_key.csv'
    item_key.to_csv(path_or_buf=file_name, columns=('id','name'), index=False)


def generate_input_data_by_item_number(start_num, num_items): 
    """Pull items by range of Index values"""
    file_name = './item_key.csv'
    all_items = pd.read_csv(file_name, skiprows=[])
    items = all_items[start_num: start_num + num_items]
    print("Item List generated by item number:")
    print(items)
    return items

def generate_input_data_by_item_name(names):  #Pull a specific items by name
    """Pull specific items by name"""
    file_name = './item_key.csv'
    data = pd.read_csv(file_name, skiprows=[])
    items = pd.DataFrame()
    for name in names:
        items = items.append(data.loc[data['name'] == name])
    items = items.reset_index().drop(labels='index',axis=1)
    print("Item List generated by name:")
    print(items)
    return items

def get_item_records_from_url(input_data, test_set_size):
    """ Iterate through all items, grab data from api and append to dataframe """
    i = 0
    item_lookup_failed = False
    item_records = pd.DataFrame(columns=['id', 'name', 'data'])
    while(i<input_data['id'].count()):
        key = input_data.iloc[i]['id']
        name = input_data.iloc[i]['name']
        #Print item information only on first data retrieval attempt 
        if not item_lookup_failed:
            print('Querying API for ' + name + ' data. ' + 'Item Id: ' + str(key))
        try:
            #Attempt to get data from API, retry if HTTP error
            url = f'https://api.rsbuddy.com/grandExchange?a=graph&g=240&start=1474615279000&i={key}'
            temp_df = pd.DataFrame()
            temp_df = pd.read_json(path_or_buf=url,orient='records', convert_axes=False)
            item_records = item_records.append({'id':key, 'name': name, 'data':temp_df}, ignore_index=True)
            i+=1
            item_lookup_failed = False
        except:
            print("Retrying...")
            time.sleep(1) #Avoid getting blacklisted by API
            item_lookup_failed = True
    print('Item retrieval complete!')

    #Add correct formatting to item record dates/times
    item_records = format_item_record_dates(item_records)

    return item_records

def format_item_record_dates(item_records):
    """ Converts timestamp to Unix seconds, tacks on formatted date field, and creates Unix seconds from most recent datapoint, and Unix seconds position from Jan-1"""
    #Iterate through each item and convert timestamps from milliseconds to seconds
    for _, row in item_records.iterrows():
        print("Formatting item record dates for: " +  row['name'] + "...")
        row['data'] = row['data'][['ts','buyingPrice','buyingCompleted','sellingPrice','sellingCompleted', 'overallPrice','overallCompleted']]
        row['data'] = row['data'].sort_values(by=['ts'],ascending=1).reset_index()
        for ind, r in row['data'].iterrows():
            r['ts'] = int(r['ts']/1000)
            row['data'].loc[ind, 'ts'] = int(r['ts'])
            
        #Refactor unix timestamps where the most recent record is 0
        timestamps = pd.DataFrame(columns=['ts','tsFromCurrent', 'tsYtd'],data=row['data']['ts'])
        lastTs = timestamps['ts'][len(timestamps)-1]
        i = 0
        print ('Finding timestamps from most recent datapoint.')
        while(i<len(timestamps)):
            timestamps['tsFromCurrent'][i] = abs(timestamps['ts'][i] - lastTs)    
            i += 1
        
        #Create a list of all years and corresponding unix since 1970 (Unix = 0)
        years = pd.DataFrame(columns=['year', 'unix'])    
        i = 0
        for year in range(2015,2050):
            unix = (year - 1970) * 31557600
            years.loc[i] = [year,unix]
            i+=1

        #Refactor unix timestamps to show time since Jan-1
        i = 0
        j = 0
        newYear = True
        while(i<len(timestamps)):
            #Iterate through each year to find all timestamps wihtin that date range
            unixYear = years.iloc[j]['unix']
            if newYear == True:
                print("Checking entries for year: " + str(years.iloc[j]['year']))
            ytd = abs(timestamps['ts'][i] - unixYear)
            if(ytd < 31557600):
                ytd = abs(timestamps.iloc[i]['ts'] - unixYear) #this may be redundant. possibly deprecate
                timestamps['tsYtd'][i] = int(ytd)
                i += 1
                newYear = False
            else: 
                print("Done with this year. Moving on to the next.")
                j += 1
                newYear = True
        #Append the new timestamps to the master item DataFrame
        for ind, r in row['data'].iterrows():
            row['data'].loc[ind, 'date'] = datetime.datetime.fromtimestamp(r['ts']).isoformat()
            row['data'].loc[ind, 'tsFromCurrent'] = timestamps.iloc[ind]['tsFromCurrent']
            row['data'].loc[ind, 'tsYtd'] = timestamps.iloc[ind]['tsYtd']            
        row['data'] = row['data'].drop(labels='index',axis=1)
    return item_records

def generate_train_and_test_deltas(item_records):
    """ 
        Iterate through each item record and generate percentage deltas for buy price, 
        buy completed, sell price, sell completed, overall price, and overall completed
        compared to last sample period. 
    """
    for _, row in item_records.iterrows():
        for col_name in ['buyingPricePer', 'buyingCompletedPer', 'sellingPricePer', 
                        'sellingCompletedPer', 'overallPricePer', 'overallCompletedPer']:
            series_length = len(row['data']['ts'])
            row['data'][col_name] = pd.Series(np.random.randn(series_length), index=row['data'].index)
        #Each row represents and item, which has an associated dataframe 
        for index, _ in row['data'].iterrows():
            #Use data from last row to calc percentage change
            for col_name in ['buyingPrice', 'buyingCompleted', 'sellingPrice', 'sellingCompleted', 
                            'overallPrice', 'overallCompleted']:
                if(index == 0):
                    row['data'].loc[index, col_name + 'Per'] = 0
                else:
                    row['data'].loc[index, col_name + 'Per'] = calculate_percentage_change(row['data'], col_name, index)
                
                
            
def calculate_percentage_change(dataframe, columnName, index):
    """Calculate delta from last time stamp"""
    return (dataframe.loc[index, columnName] - dataframe.loc[index-1, columnName])/dataframe.loc[index-1, columnName]


def createTrainAndTestSet(item_records):
    print("Creating Test and Training Datasets...")
    #create test_item object to log training and test sets
    test_item = pd.DataFrame(columns=['id', 'name', 'train_val', 'test_val', 'train_per', 'test_per', 'pred_per'])
    record_index = 0
    #iterate through the urlquery results to generate training sets
    while(record_index < item_records['id'].count()):  
        item_id = item_records.iloc[record_index]['id'] 
        print("Creating Datasets for item ID: " + str(item_id))
        
        #takes the item_id and generates the test data for the specified parameters as an array
        test_set = item_records.loc[item_records['id'] == item_id]
        
        #for column key, uncomment this line below:
        #train_x_headers = list(urlquery.iloc[0]['data'].columns.values[0:6])
        #print(train_x_headers)

        #create training datasets
        train_val = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,0:6]
        print ("The shape of the " + test_set.iloc[0]['name'] +  " input array is: "+ str(train_val.shape))

        #create test set to estimate next 6 prices (next day of prices)
        test_val = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,0:6]

        #Generate deltas
        generate_train_and_test_deltas(item_records)

        #test key to compare to Y-hat
        test_item = test_item.append({'id':item_id, 'name': test_set.iloc[0]['name'], 'train_val': train_val, 'test_val': test_val}, ignore_index=True)
        print("Record completed at test_item position [" + str(record_index) + "]")
        record_index += 1

    return test_item

    #TODO: Create training and test sets using percentages
    #TODO: Test model size


    #random shit
    #import keras.backend as K
    """
    train_val = test_item.iloc[2]['train_val'] # test data. not real
    train_y = test_item.iloc[2]['train_y'] # test data. not real

    
    def create_model(layers,activation):
        model = Sequential()
        for i, nodes in enumerate(layers):
            if i==0:
                model.add(Dense(nodes,input_dim=train_val.shape[1]))
                model.add(Activation(activation))
            else: 
                model.add(Dense(nodes))
                model.add(Activation(activation))
        model.add(Dense(1)) #Note: no activations present beyond this point

        model.compile(optimizer='adadelta', loss='mse')
        return model

    print('creating model')
    model = KerasRegressor(build_fn=create_model, verbose = 0)
    layers = [[16], [4,2], [4], [16,4]]
    activations = ['tanh', 'relu']
    param_grid = dict(layers=layers, activation=activations, batch_size = [42, 180], epochs=[6])
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error')

    # grid_result = grid.fit(train_x, train_y) testing for network size """

In [2]:
if __name__ == '__main__':
    #Get a list of all items
    generate_item_records_from_summary()

    #Pull items by range of Index values
    #input_data = generate_input_data_by_item_number(0,3)
    
    #Pull specific items by name
    input_data = generate_input_data_by_item_name(['Bandos chestplate','Dragon boots', 'Rune arrow'])

    #In Days - define the number of epochs to include in the test data
    test_set_size = 30
    item_records = get_item_records_from_url(input_data, test_set_size)

    #Create test_item object to log training and test sets
    test_item = createTrainAndTestSet(item_records)

    #Test creation of Neural Net
    nn = NeuralNetwork()
    nn.generateInitialNetwork([1,1])
    nn.calculateNodeActivations()
    print(nn.outputNode.getActivation())

Item List generated by name:
      id               name
0  11832  Bandos chestplate
1  11840       Dragon boots
2    892         Rune arrow
Querying API for Bandos chestplate data. Item Id: 11832
Querying API for Dragon boots data. Item Id: 11840
Querying API for Rune arrow data. Item Id: 892
Item retrieval complete!
Formatting item record dates for: Bandos chestplate...
Finding timestamps from most recent datapoint.
Checking entries for year: 2015
Done with this year. Moving on to the next.
Checking entries for year: 2016
Done with this year. Moving on to the next.
Checking entries for year: 2017
Done with this year. Moving on to the next.
Checking entries for year: 2018
Formatting item record dates for: Dragon boots...
Finding timestamps from most recent datapoint.
Checking entries for year: 2015
Done with this year. Moving on to the next.
Checking entries for year: 2016
Done with this year. Moving on to the next.
Checking entries for year: 2017
Done with this year. Moving on to the

In [9]:
item_records.iloc[0]['data']

,ts,buyingPrice,buyingCompleted,sellingPrice,sellingCompleted,overallPrice,overallCompleted,date,tsFromCurrent,tsYtd,buyingPricePer,buyingCompletedPer,sellingPricePer,sellingCompletedPer,overallPricePer,overallCompletedPer
0,1474603200,12856989.0,133.0,12839326.0,145.0,12847776,278,2016-09-22T21:00:00,59875200.0,22953600.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1474617600,12843079.0,82.0,12827839.0,70.0,12836061,152,2016-09-23T01:00:00,59860800.0,22968000.0,-0.001082,-0.383459,-0.000895,-0.517241,-0.000912,-0.453237
2,1474632000,12854836.0,132.0,12854102.0,124.0,12854480,256,2016-09-23T05:00:00,59846400.0,22982400.0,0.000915,0.609756,0.002047,0.771429,0.001435,0.684211
3,1474646400,12849086.0,132.0,12828015.0,141.0,12838203,273,2016-09-23T09:00:00,59832000.0,22996800.0,-0.000447,0.000000,-0.002029,0.137097,-0.001266,0.066406
4,1474660800,12839487.0,172.0,12824031.0,186.0,12831457,358,2016-09-23T13:00:00,59817600.0,23011200.0,-0.000747,0.303030,-0.000311,0.319149,-0.000525,0.311355
5,1474675200,12846131.0,142.0,12827990.0,142.0,12837061,284,2016-09-23T17:00:00,59803200.0,23025600.0,0.000517,-0.174419,0.000309,-0.236559,0.000437,-0.206704
6,1474689600,12855120.0,123.0,12847313.0,120.0,12851264,243,2016-09-23T21:00:00,59788800.0,23040000.0,0.000700,-0.133803,0.001506,-0.154930,0.001106,-0.144366
7,1474704000,12857213.0,105.0,12847855.0,105.0,12852534,210,2016-09-24T01:00:00,59774400.0,23054400.0,0.000163,-0.146341,0.000042,-0.125000,0.000099,-0.135802
8,1474718400,12815017.0,136.0,12787150.0,143.0,12800734,279,2016-09-24T05:00:00,59760000.0,23068800.0,-0.003282,0.295238,-0.004725,0.361905,-0.004030,0.328571
9,1474732800,12845882.0,172.0,12827978.0,157.0,12837338,329,2016-09-24T09:00:00,59745600.0,23083200.0,0.002409,0.264706,0.003193,0.097902,0.002860,0.179211


<font size="+3"><b>Testing of network size:</b></font>

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#import keras.backend as K
train_x = test_item.iloc[2]['train_x'] # test data. not real
 
def create_model(layesr,activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=train_x.shape[1]))
            model.add(Activation(activation))
        else: 
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) #Note: no activations present beyond this point

    model.compile(optimizer='adadelta', loss='mse')
    return model

model = KerasRegressor(build_fn=create_model, verbose = 0)

In [ ]:
layers = [[16], [4,2], [4], [16,4]]
activations = [tanh, relu]
param_grid = dict(layers=layers, activation=activations, batch_size = [42, 180], epochs[6])
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error')